In [3]:
from PIL import Image
from io import BytesIO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import pandas as pd

In [2]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = '/opt/homebrew/bin/tesseract' 

In [5]:
import cv2
import pytesseract
import pandas as pd
import os
import re

# Define the directory path
dir_path = "/Users/shivanipatel/Downloads/archive/yolo x image/yolo x image"

# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Image_Name', 'Routing_Number', 'Account_Number', 'Check_Number', 'Formatted_MICR_Line'])

# Function to clean the extracted text
def clean_text(text):
    if text not in [' ', '  ', '']:
        text = re.sub('[^A-Za-z0-9-/,.() ]+', '', text)
        text = text.strip()
        text = re.sub(r'\s{2,}', ' ', text)
    return text

# Function to extract the MICR line
def extract_micr_line(text):
    micr_pattern = re.compile(r'C\d{6}C\d{9}A\d{6}C\d{2}')
    match = micr_pattern.search(text.replace(" ", "").replace("\n", ""))
    return match.group() if match else None

# Function to format the MICR line
def format_micr_line(micr_line):
    return re.sub(r'[CA]', '', micr_line)

# Function to parse the MICR code
def parse_micr(micr_code):
    micr_code = re.sub(r'\D', '', micr_code)
    routing_number = micr_code[:9]
    check_number = micr_code[-4:]
    account_number = micr_code[9:-4]
    return routing_number, account_number, check_number

# Function to format MICR for printing
def format_micr_for_print(routing_number, account_number, check_number):
    return "{} {} {}".format(routing_number, account_number, check_number)

# Loop over each file in the directory
for filename in os.listdir(dir_path):
    if filename.lower().endswith((".jpg", ".png")):
        file_path = os.path.join(dir_path, filename)
        img = cv2.imread(file_path)

        # Preprocess the image
        img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        micr_img = img[420:,:]  # Adjust this slicing based on where the MICR line is in your images
        e13bb_text = pytesseract.image_to_string(micr_img, lang='e13bb')
        e13bb_clean = clean_text(e13bb_text)

        # Extract and process MICR line
        micr_line = extract_micr_line(e13bb_clean)
        if micr_line:
            formatted_micr = format_micr_line(micr_line)
            routing_number, account_number, check_number = parse_micr(formatted_micr)
            formatted_micr_line = format_micr_for_print(routing_number, account_number, check_number)

            # Create a DataFrame for the current image's results
            current_results = pd.DataFrame({
                'Image_Name': [filename],
                'Routing_Number': [routing_number],
                'Account_Number': [account_number],
                'Check_Number': [check_number],
                'Formatted_MICR_Line': [formatted_micr_line]
            })

            # Concatenate the current results with the main DataFrame
            results_df = pd.concat([results_df, current_results], ignore_index=True)

# Save the results to a CSV file
csv_file_path = '/Users/shivanipatel/Downloads/archive/yolo x image/micr_results.csv'  # Replace with your desired path
results_df.to_csv(csv_file_path, index=False)

print(f"MICR extraction completed. Results saved to {csv_file_path}")


MICR extraction completed. Results saved to /Users/shivanipatel/Downloads/archive/yolo x image/micr_results.csv
